# Notebook for submission data to BioSamples

In [1]:
import subprocess
import json
import os

### 1 (required). Please make sure to signup in AAP webpage (https://aai.ebi.ac.uk/registerUser), then provide your username and passwork in next cell

In [2]:
your_aap_username = ""
your_aap_password = ""

### 2 (required). Fetching your token. Keep in mind that the token is valid for 1 hour. After that you need to fetch new one.

In [3]:
cmd = f"curl -u {your_aap_username}:{your_aap_password} https://api.aai.ebi.ac.uk/auth"
proc = subprocess.Popen([cmd], stdout=subprocess.PIPE, shell=True)
(token, err) = proc.communicate()
token = token.decode("utf-8")

### 3 (optional). Create team. Please provide 'description' - a description of your team (optional) and 'centre_name' - name of your institute or project (required)

In [4]:
description = "My lab group"
centre_name = "An Institute"
team_json = {
    "description": description,
    "centreName": centre_name
}
team_json = json.dumps(team_json)

In [5]:
cmd = f"curl 'https://submission.ebi.ac.uk/api/user/teams' -i -X POST -H 'Content-Type: application/json' -H 'Accept: application/hal+json' -H 'Authorization: Bearer {token}' -d '{team_json}'"
proc = subprocess.Popen([cmd], stdout=subprocess.PIPE, shell=True)
(out, err) = proc.communicate()

### 4 (optional). Managing your team

In [6]:
# Find your user reference
cmd = f"curl https://api.aai.ebi.ac.uk/users/{your_aap_username} -H 'Authorization: Bearer {token}'"
proc = subprocess.Popen([cmd], stdout=subprocess.PIPE, shell=True)
(out, err) = proc.communicate()
out = json.loads(out.decode('utf-8'))
user_reference = out['userReference']

# Find your domain reference
cmd = f"curl https://api.aai.ebi.ac.uk/my/domains -H 'Authorization: Bearer {token}'"
proc = subprocess.Popen([cmd], stdout=subprocess.PIPE, shell=True)
(out, err) = proc.communicate()
out = json.loads(out.decode('utf-8'))
# Here we use 0 to get first team from list, adjust this value to get team you need
domain_reference = out[0]['domainReference']

# Adding user to a team
cmd = f"curl 'https://api.aai.ebi.ac.uk/domains/{domain_reference}/{user_reference}/user' -X PUT -H 'Authorization: Bearer {token}' -H 'Accept: application/hal+json'"
proc = subprocess.Popen([cmd], stdout=subprocess.PIPE, shell=True)
(out, err) = proc.communicate()
out = json.loads(out.decode('utf-8'))
print(out)
# Each token includes all the details about who you are, and which teams you are a member of. You have now created 
# a new team and added yourself to it since you requested your first token, so that first token is out of date. You 
# should request a new one, as you did before. The new one will include the team you just created. You can use this 
# new token to access the submissions API.

KeyError: 'userReference'